In [24]:
# Nhiệm vụ 4 – Merge dữ liệu

import pandas as pd
import numpy as np

In [25]:
# 1. Tải dữ liệu
df_owner = pd.read_csv('owner_info_sach.csv')
df_vehicle = pd.read_csv('vehicle_registration_sach.csv')
df_emission = pd.read_csv('emission_test_sach.csv')

# Chuyển đổi cột ngày tháng sang kiểu datetime để đảm bảo tính nhất quán
df_vehicle['reg_date'] = pd.to_datetime(df_vehicle['reg_date'])
df_emission['test_date'] = pd.to_datetime(df_emission['test_date'])

# 2. Hợp nhất 1: owner_info và vehicle_registration (Thêm thông tin chủ sở hữu/tỉnh)
# Sử dụng left merge để giữ lại tất cả các phương tiện
df_vehicle_owner = pd.merge(
    df_vehicle,
    df_owner,
    on='owner_id',
    how='left'
)

# 3. Hợp nhất 2: Thêm emission_test (kết quả kiểm định khí thải)
# Sử dụng full outer merge để có thể phát hiện các trường hợp thiếu/thừa dữ liệu ở cả hai bên
df_merged_full = pd.merge(
    df_vehicle_owner,
    df_emission,
    on='plate_no',
    how='outer',
    suffixes=('_reg', '_test') # Thêm hậu tố để phân biệt các cột trùng tên (nếu có)
)

# Lưu file dữ liệu hoàn chỉnh để phục vụ phân tích sâu hơn
df_merged_full.to_csv('merged_full_data.csv', index=False)

print("Đã tạo DataFrame hợp nhất hoàn chỉnh: df_merged_full")
print("\n5 dòng đầu của DataFrame hợp nhất:")
print(df_merged_full.head())
print("\nThông tin tổng quan về DataFrame hợp nhất:")
print(df_merged_full.info())

Đã tạo DataFrame hợp nhất hoàn chỉnh: df_merged_full

5 dòng đầu của DataFrame hợp nhất:
     plate_no owner_id vehicle_type  engine_cc   reg_date    owner_name  \
0  51F-899.26   OWN136     Ô tô con        250 2021-03-24   Hoang Van E   
1  51F-881.31   OWN123       Xe máy        150 2021-10-11    Pham Thi D   
2  30A-009.15   OWN150     Ô tô con        100 2022-11-21    Pham Thi D   
3  30A-889.81   OWN141       Xe máy        250 2022-06-07      Le Van C   
4  43A-309.65   OWN103       Xe máy        125 2022-09-26  Nguyen Van A   

     province  test_date     result  co_level  
0  Bình Dương 2022-02-09  Không đạt       0.3  
1     Đà Nẵng 2023-02-08  Không đạt       0.3  
2      Hà Nội        NaT        NaN       NaN  
3      Hà Nội        NaT        NaN       NaN  
4      Hà Nội 2022-06-05        Đạt       0.5  

Thông tin tổng quan về DataFrame hợp nhất:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 10 columns):
 #   Column        Non-

In [26]:
# Phát hiện các phương tiện chưa có dữ liệu kiểm định
# Xác định các phương tiện chưa có dữ liệu kiểm định
vehicles_no_test = df_final[df_final['_merge'] == 'left_only'].copy()

print(f"\n1. Tổng số phương tiện chưa có dữ liệu kiểm định: {vehicles_no_test['plate_no'].nunique()}")

if len(vehicles_no_test) > 0:
    print("\n2. Chi tiết các phương tiện chưa có dữ liệu kiểm định:")
    vehicles_no_test_summary = vehicles_no_test.groupby('plate_no').agg({
        'owner_id': 'first',
        'owner_name': 'first',
        'province': 'first',
        'vehicle_type': 'first',
        'engine_cc': 'first',
        'reg_date': 'first'
    }).reset_index()
    
    print(f"\n   Danh sách {min(20, len(vehicles_no_test_summary))} phương tiện đầu tiên:")
    print(vehicles_no_test_summary.head(20))
    
    

else:
    print("\n   ✓ Tất cả phương tiện đều có dữ liệu kiểm định!")



1. Tổng số phương tiện chưa có dữ liệu kiểm định: 57

2. Chi tiết các phương tiện chưa có dữ liệu kiểm định:

   Danh sách 20 phương tiện đầu tiên:
      plate_no owner_id    owner_name    province vehicle_type  engine_cc  \
0   29A-072.97   OWN114    Tran Thi B      Hà Nội     Ô tô con        100   
1   29A-104.24   OWN136   Hoang Van E  Bình Dương       Xe tải        125   
2   29A-115.61   OWN144    Tran Thi B  Bình Dương       Xe máy        150   
3   29A-233.27   OWN134      Le Van C     Đà Nẵng       Xe tải        110   
4   29A-269.31   OWN109      Le Van C     Đà Nẵng       Xe tải        125   
5   29A-299.73   OWN142    Pham Thi D     Đà Nẵng       Xe máy        250   
6   29A-349.60   OWN134      Le Van C     Đà Nẵng       Xe máy        100   
7   29A-355.09   OWN146    Tran Thi B      Hà Nội     Ô tô con        100   
8   29A-356.43   OWN131    Tran Thi B      TP.HCM       Xe tải        125   
9   29A-396.59   OWN100    Tran Thi B      TP.HCM       Xe tải        100   
10  

In [27]:

# Phát hiện các phương tiện có nhiều bản ghi kiểm định

print("=== PHÁT HIỆN PHƯƠNG TIỆN CÓ NHIỀU BẢN GHI KIỂM ĐỊNH ===")

# Đếm số lượng bản ghi kiểm định cho mỗi phương tiện
test_counts = emission_test.groupby('plate_no').size().reset_index(name='so_lan_kiem_dinh')
test_counts = test_counts.sort_values('so_lan_kiem_dinh', ascending=False)

print(f"\n1. Thống kê số lần kiểm định:")
print(f"   - Tổng số phương tiện có kiểm định: {len(test_counts)}")
print(f"   - Số lần kiểm định trung bình: {test_counts['so_lan_kiem_dinh'].mean():.2f}")
print(f"   - Số lần kiểm định tối đa: {test_counts['so_lan_kiem_dinh'].max()}")
print(f"   - Số lần kiểm định tối thiểu: {test_counts['so_lan_kiem_dinh'].min()}")

# Phân bố số lần kiểm định
print(f"\n2. Phân bố số lần kiểm định:")
distribution = test_counts['so_lan_kiem_dinh'].value_counts().sort_index()
for count, frequency in distribution.items():
    print(f"   - {count} lần: {frequency} phương tiện")

# Phát hiện các phương tiện có nhiều bản ghi (>= 2 lần)
vehicles_multiple_tests = test_counts[test_counts['so_lan_kiem_dinh'] >= 2]
print(f"\n3. Phương tiện có từ 2 lần kiểm định trở lên: {len(vehicles_multiple_tests)}")

if len(vehicles_multiple_tests) > 0:
    print(f"\n   Top 10 phương tiện có nhiều lần kiểm định nhất:")
    top_vehicles = vehicles_multiple_tests.head(10)
    print(top_vehicles)
    
    # Chi tiết các phương tiện có nhiều lần kiểm định
    print(f"\n4. Chi tiết các phương tiện có nhiều lần kiểm định:")
    for plate in vehicles_multiple_tests.head(10)['plate_no']:
        vehicle_tests = emission_test[emission_test['plate_no'] == plate].sort_values('test_date')
        print(f"\n   Biển số: {plate}")
        print(f"   Số lần kiểm định: {len(vehicle_tests)}")
        print(f"   Các ngày kiểm định:")
        for idx, row in vehicle_tests.iterrows():
            print(f"     - {row['test_date']}: {row['result']} (CO: {row['co_level']})")
else:
    print("\n   ✓ Không có phương tiện nào có nhiều hơn 1 lần kiểm định")


=== PHÁT HIỆN PHƯƠNG TIỆN CÓ NHIỀU BẢN GHI KIỂM ĐỊNH ===

1. Thống kê số lần kiểm định:
   - Tổng số phương tiện có kiểm định: 63
   - Số lần kiểm định trung bình: 1.43
   - Số lần kiểm định tối đa: 4
   - Số lần kiểm định tối thiểu: 1

2. Phân bố số lần kiểm định:
   - 1 lần: 39 phương tiện
   - 2 lần: 22 phương tiện
   - 3 lần: 1 phương tiện
   - 4 lần: 1 phương tiện

3. Phương tiện có từ 2 lần kiểm định trở lên: 24

   Top 10 phương tiện có nhiều lần kiểm định nhất:
      plate_no  so_lan_kiem_dinh
28  30A-797.49                 4
7   29A-358.76                 3
31  43A-146.22                 2
16  29A-884.44                 2
38  43A-997.93                 2
44  51F-459.51                 2
46  51F-487.31                 2
33  43A-309.65                 2
47  51F-488.03                 2
29  30A-934.69                 2

4. Chi tiết các phương tiện có nhiều lần kiểm định:

   Biển số: 30A-797.49
   Số lần kiểm định: 4
   Các ngày kiểm định:
     - 2022-07-15 00:00:00: Đạt (CO: 0.5

In [28]:

# NHIỆM VỤ 5: Phát hiện các bản ghi emission_test không tìm thấy trong vehicle_registration (biển số lạ)


print("=== PHÁT HIỆN BIỂN SỐ LẠ (KHÔNG CÓ TRONG VEHICLE_REGISTRATION) ===")

# Tìm các biển số trong emission_test nhưng không có trong vehicle_registration
strange_plates = emission_test[~emission_test['plate_no'].isin(vehicle_registration['plate_no'])].copy()

print(f"\n1. Tổng số bản ghi emission_test không tìm thấy trong vehicle_registration: {len(strange_plates)}")
print(f"   Số biển số duy nhất: {strange_plates['plate_no'].nunique()}")

if len(strange_plates) > 0:
    print("\n2. Danh sách các biển số lạ:")
    strange_plates_summary = strange_plates.groupby('plate_no').agg({
        'test_date': ['count', 'min', 'max'],
        'result': lambda x: ', '.join(x.unique()),
        'co_level': 'mean'
    }).reset_index()
    strange_plates_summary.columns = ['plate_no', 'so_lan_kiem_dinh', 'ngay_kiem_dinh_dau', 'ngay_kiem_dinh_cuoi', 'ket_qua', 'co_level_tb']
    strange_plates_summary = strange_plates_summary.sort_values('so_lan_kiem_dinh', ascending=False)
    print(strange_plates_summary)
    
    print("\n3. Chi tiết từng biển số lạ:")
    for plate in strange_plates['plate_no'].unique():
        plate_tests = strange_plates[strange_plates['plate_no'] == plate].sort_values('test_date')
        print(f"\n   Biển số: {plate}")
        print(f"   Số lần kiểm định: {len(plate_tests)}")
        print(f"   Các lần kiểm định:")
        for idx, row in plate_tests.iterrows():
            print(f"     - Ngày: {row['test_date']}, Kết quả: {row['result']}, CO: {row['co_level']}")
    
    print("\n4. Phân tích kết quả kiểm định của các biển số lạ:")
    result_analysis = strange_plates['result'].value_counts()
    print(result_analysis)
    
    print("\n5. Thống kê CO level của các biển số lạ:")
    print(f"   - Trung bình: {strange_plates['co_level'].mean():.2f}")
    print(f"   - Tối đa: {strange_plates['co_level'].max():.2f}")
    print(f"   - Tối thiểu: {strange_plates['co_level'].min():.2f}")
else:
    print("\n   ✓ Tất cả biển số trong emission_test đều có trong vehicle_registration!")


=== PHÁT HIỆN BIỂN SỐ LẠ (KHÔNG CÓ TRONG VEHICLE_REGISTRATION) ===

1. Tổng số bản ghi emission_test không tìm thấy trong vehicle_registration: 0
   Số biển số duy nhất: 0

   ✓ Tất cả biển số trong emission_test đều có trong vehicle_registration!
